## General work flow for Analysing ESRF EDF files

Created in the FALL OF 2025

@authors: Edem and Brinthan (Dresselhaus-Marais Research Group, Stanford University)

This script will explore the EDF files and extract the relevant information. 

It can handle rocking, mosa, and strain layer data and plot 1D and 2D COM plots
Additionally it can convert EDF to H5 files for easier analysis and resolve the 3D volume


## Import essential Libraries

In [ ]:


## Import essential Libraries 

import numpy as np
import os
import re
#import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import datetime
import scienceplots
plt.style.use('science')
# from image_processing import *
import imageio



def log_message(message, log_file= '/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_remountedafterheating/Mg_remountedafterheating_farfield/results.txt'):
    with open(log_file, 'a') as f:
        f.write(message + '\n')

In [ ]:
# Extract the files 

import os
import tarfile

# Extract mosa_layer scans
def extract_tar(tar_dir, extract_dir, prefix):
    #prefix = 'mosalayer_10x_'
    for tar_file in os.listdir(tar_dir):
        if tar_file.startswith(prefix) and tar_file.endswith('.tar.gz'):
            print(f'Extracting {tar_file}...')
            file_path = os.path.join(tar_dir, tar_file)
            with tarfile.open(file_path, 'r:gz') as tar:
                tar.extractall(extract_dir)
tar_dir_rocking = '/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_101/Mg101_farfield_rt/'
extract_dir_rocking = '/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_101/Mg101_farfield_rt/Before_heating_extracted/'
prefix_rocking = 'DEPow_0.000000_rockinglayer_10x_'
extract_tar(tar_dir_rocking, extract_dir_rocking, prefix_rocking)

In [ ]:
# this functions will explore and read edf files 

def edf_info(filename):
    edfheader = {}
    pattern = r'(?P<parameter>.*)=(?P<value>.*);'
    try:
        with open(filename, 'r', encoding='unicode_escape') as file:
            while True:
                line = file.readline()
                if not line:
                    break  
                if line.strip() == 'Detector  = frelon4m (sn=29) ;':
                    line = 'Detector  = frelon4m (sn 29) ;'
                match = re.match(pattern, line)
                if match:
                    parameter = match.group('parameter').strip().lower()
                    value = match.group('value').strip()
                    edfheader[parameter] = value
                if '}' in line:
                    current_pos = file.tell()
                    header_length = current_pos
                    if header_length % 512 != 0: # I dont know how I managed to find the header length as the multiplication of 512, I did this around Jan 2024. 
                        log_message(f'Header is not a multiple of 512 bytes in {filename}!')
                        log_message('Setting header length correctly!')
                        header_length = round(header_length / 512) * 512
                    edfheader['headerlength'] = header_length
                    break
    except UnicodeDecodeError:
        log_message(f"Error decoding file: {filename}")
        return None

    if 'headerlength' not in edfheader:
        raise ValueError(f"Header length not found in {filename}.")
        
    return edfheader

def motor_positions(filename):

    info = edf_info(filename)

    motor_mne = info.get('motor_mne', '').split()
    motor_pos = info.get('motor_pos', '').split()
    motor_pos = [float(value) for value in motor_pos]

    motor_info = {}
    for mne, pos in zip(motor_mne, motor_pos):
        motor_info[mne] = pos

    return motor_info


def edf_read(filename):
    info = edf_info(filename)
    if info is None:
        log_message(f"Skipping file due to read error: {filename}")
        return None, None

    try:
        with open(filename, 'rb') as fid:
            # log_message(f"Reading {filename}...")
            fid.seek(info['headerlength'])
            image = np.fromfile(fid, dtype=np.uint16)
        
        dim_2 = int(info['dim_2'])
        dim_1 = int(info['dim_1'])
        image = image.reshape((dim_2, dim_1))
        return image, info
    except Exception as e:
        log_message(f"Error reading file {filename}: {e}")
        return None, None

In [ ]:
# function to extract the data and real space images from one layer
# You can select any layer you want, I preferably go with the middle layer of the scan

def extract(path):
    data = {}
    
    file_list = [file_name for file_name in os.listdir(path) if file_name.endswith('.edf')]
    
    for file_name in file_list:
        file_path = os.path.join(path, file_name)
        image_data, info = edf_read(file_path)

        if info is None:
            log_message(f"Skipping file due to read error: {file_path}")
            continue

        motor_mne = info.get('motor_mne', '').split()
        motor_pos = [float(value) for value in info.get('motor_pos', '').split()]
        motor_info = dict(zip(motor_mne, motor_pos))

        # could be useful to add the descriptions for the names of the motors here
        
        keys_to_extract = [
            'DECurr', 'DEPow', 'DEVolt', 'auxx', 'auxy', 'auxz', 'bstop', 
            'cdpitch', 'cdx', 'cdy', 'cdyaw', 'cdz', 'chi', 'corx', 'corz', 
            'dcx', 'dcy', 'dcz', 'decoh', 'delVsp', 'detx', 'dety', 'detz', 
            'diffry', 'diffrz', 'diffty', 'difftz', 'euro_sp', 'fffoc1', 
            'fffoc2', 'ffpitch', 'ffrotc', 'ffsel', 'ffy', 'ffz', 'furny', 
            'hfoc', 'hrotc', 'htth', 'hxpitch', 'hxroll', 'hxx', 'hxy', 
            'hxyaw', 'hxz', 'icx', 'lenssel', 'mainx', 'mono', 'mrot', 
            'nffoc', 'nfrotc', 'nfrz', 'nfx', 'nfy', 'nfz', 'obpitch', 
            'obx', 'oby', 'obyaw', 'obz', 'pcofoc', 'phi', 'phpy', 
            'phpz', 'py', 'pz', 's3hg', 's3ho', 's3vg', 's3vo', 
            's4hg', 's4ho', 's4vg', 's4vo', 's5hg', 's5ho', 's5vg', 
            's5vo', 's6hg', 's6ho', 's6vg', 's6vo', 's7hg', 's7ho', 
            's7vg', 's7vo', 'smx', 'smy', 'smz', 'sptlens', 'sptmemb', 
            'unused', 'x2z'
        ]

        extracted_data = {key: motor_info.get(key) for key in keys_to_extract}

        # Get frame number, handling different keys based on file type
        # frame_no = info.get('run')  # or check for 'acq_frame_nb' if needed
        frame_no = info.get('acq_frame_nb')
        time_of_frame = info.get('time_of_frame')
        time_of_frame = float(time_of_frame)
        # problem with the saving the images is we will loose the metadata. 

        # image data is available here as image_data. This should be the ideal place to make any preprocessing to the image data. So that the image data along with the metadata can be used for further analysis.

        # preprocessing steps - deducting the darks, noise removal, shift correction, and selection of ROI (got this from Darfix video)

        # need to do dark subraction before doing statistics on the image data

        # problem with doing the normalisation is that the image will be scaled to 0-1, then returning the max intensity will not be useful, but the mean intensity will be useful

        # the image data type is int16: which means the intensity values are between -32768 to 32767

        max_intensity = np.max(image_data)
        image_data = image_data

        # max_intensity, mean_intensity, image_data_treated = image_processing(image_data)

        # need to log basic image information as well, but now now - this can be then integrated to the image_processing function

        # I can add the image data to the dictionary if needed, but skipping for now

        # this data dictionary should be the master dictionary that contains all the metadata and the image data for all the files in the directory

        # extracted_key_value = data[file_name]['extracted_key'] - this is how you can access the extracted data for a specific file, therefore this might be a good place to add the image data to the dictionary

        # even for the hdf5 files, if we can get the data in this format, i.e., like a dictionary, it will be easy for the further processing 
        data[file_name] = {
            'Frame No': frame_no,
            'intensity': max_intensity,
            'time_of_frame': time_of_frame,
            'image_data': image_data,
            **extracted_data
        }

    return data, motor_mne, motor_pos

#Example: We extracting the data for one layer for this mosa scan

path_1 = '/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_remountedafterheating/Mg_remountedafterheating_farfield/Mosa_layer_extracted/mosalayer_10x_20'
data_1 = extract(path_1)
data_1

In [ ]:
# function to sort out data and see how phi and chi are varying. 
#P.S: diffry = phi and chi = phi. It is a bit confusing but take this convention as it is for now.

from collections import OrderedDict

# Sort by phi and diffry
sorted_data = OrderedDict(
    sorted(data_1.items(), 
           key=lambda x: (x[1]['phi'], x[1]['diffry'],x[1]['intensity']))
)

# Print sorted order
print("Sorted Data:")
for idx, (filename, values) in enumerate(sorted_data.items()):
    print(f"{idx}: phi={values['phi']:.4f}, diffry={values['diffry']:.4f}, "
          f"frame={values['Frame No']}, intensity = {values['intensity']:.4f} file={filename}")

In [ ]:
# plot to see how they vary: 

import matplotlib.pyplot as plt
import numpy as np
# Seems jupyter can't read latex. Turning off latex
plt.rcParams["text.usetex"] = False  # Turn off LaTeX

# Extract diffry and phi values from sorted_data
diffry_values = [values['diffry'] for filename, values in sorted_data.items()]
phi_values = [values['phi'] for filename, values in sorted_data.items()]

# Create plots
plt.figure(figsize=(10, 3))

plt.subplot(1, 2, 1)
plt.plot(diffry_values, 'b', markersize=1)
plt.title('Phi Progression')
plt.xlabel('Index')
plt.ylabel('Diffry (degrees)')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(phi_values, 'r', markersize=1)
plt.title('Chi Progression')
plt.xlabel('Index')
plt.ylabel('Phi (degrees)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate mu_length (diffry_length) and phi_length
diffry_array = np.array(diffry_values)
diffry_length = int(np.argwhere(diffry_array == np.max(diffry_array))[0]) + 1
phi_length = int(len(diffry_array) / diffry_length)

print(f"diffry_length: {diffry_length}")
print(f"phi_length: {phi_length}")

In [ ]:
#Function to plot real-space images
#This function also outputs rocking curves, mosa, and strain maps for one layer 


def plot_images_for_folder(data, output_folder, start_frame=0, end_frame=31):
    frame_numbers = sorted(set([int(value['Frame No']) for key, value in data.items()]))
    
    if start_frame is not None:
        frame_numbers = [f for f in frame_numbers if f >= start_frame]
    if end_frame is not None:
        frame_numbers = [f for f in frame_numbers if f <= end_frame]
    
    num_columns = 5
    num_rows = (len(frame_numbers) + num_columns - 1) // num_columns
    fig, axs = plt.subplots(num_rows, num_columns, figsize=(num_columns * 5, num_rows * 5))
    fig.tight_layout(pad=3.0)
    axs = axs.flatten()

    rocking_diffry_values = []  
    rocking_intensity_values = [] 
    mosa_diffry_values = []  
    mosa_intensity_values = []  
    mosaic_phi_values = []
    strain_diffry_values = []  
    strain_intensity_values = [] 
    strain_obpitch_values = []

    for idx, frame_no in enumerate(frame_numbers):
        for file_name, value in data.items():
            if value['Frame No'] == str(frame_no):

                image_data = value['image_data']
                diffry = value.get('diffry', 'N/A')
                phi = value.get('phi', 'N/A')
                obpitch = value.get('obpitch', 'N/A')
                intensity = value['intensity']

                # roi
                # image_data = image_data[100:400, 100:400]
                intensity = np.max(image_data)

                if "rocking" in file_name.lower():
                    axs[idx].imshow(image_data, vmin=np.percentile(image_data, 1), vmax=np.percentile(image_data, 99))
                    axs[idx].set_title(f"Rocking - Frame: {frame_no}\nDiffry: {diffry}, Intensity: {intensity}")
                    rocking_diffry_values.append(diffry)  
                    rocking_intensity_values.append(intensity)  
                elif "mosa" in file_name.lower():
                    axs[idx].imshow(image_data, vmin=np.percentile(image_data, 5), vmax=np.percentile(image_data, 99.5))
                    axs[idx].set_title(f"Mosa - Frame: {frame_no}\nDiffry: {diffry:.4f}, Intensity: {intensity}, Phi: {phi:.4f}")
                    mosa_diffry_values.append(diffry)  
                    mosa_intensity_values.append(intensity)  
                    mosaic_phi_values.append(phi)
                elif "strain" in file_name.lower():
                    axs[idx].imshow(image_data, vmin=np.percentile(image_data, 5), vmax=np.percentile(image_data, 99.5))
                    axs[idx].set_title(f"Strain - Frame: {frame_no}\nDiffry: {diffry}, Intensity: {intensity}, Obpitch: {obpitch}")
                    strain_diffry_values.append(diffry)  
                    strain_intensity_values.append(intensity)  
                    strain_obpitch_values.append(obpitch)
                else:
                    log_message(f"Unknown file type: {file_name}")

                axs[idx].axis('off')
                break
        else:
            axs[idx].set_visible(False)
            print(f"No data returned for frame {frame_no}")

    for idx in range(len(frame_numbers), len(axs)):
        axs[idx].set_visible(False)

    frame_range = f"frames_{start_frame or 'start'}_to_{end_frame or 'end'}"
    plt.savefig(f'{output_folder}/all_{frame_range}.png', dpi=300)
    log_message(f"Saved frames subplot as {output_folder}/all_{frame_range}.png")
    plt.close()

    # Rocking curve plot (normalized diffry)
    if rocking_diffry_values and rocking_intensity_values:
        diffry_array = np.array(rocking_diffry_values)
        intensity_array = np.array(rocking_intensity_values)
        
        # Normalize diffry
        diffry_median = np.median(diffry_array)
        diffry_normalized = diffry_array - diffry_median
        
        fig, ax = plt.subplots(1, 1, figsize=(10, 5))
        ax.scatter(diffry_normalized, intensity_array, s=5, c='r') 
        ax.set_title(f'Rocking Curve\n(Centered at Diffry={diffry_median:.4f}°)')
        ax.set_xlabel('Diffry (normalized, degrees)')
        ax.set_ylabel('Intensity')
        ax.grid(True, alpha=0.3)
        plt.savefig(f'{output_folder}/diffry_vs_intensity.png', dpi=300)
        log_message(f"Saved {output_folder}/diffry_vs_intensity.png")
        plt.close()
        
        print(f"\nRocking curve - Diffry normalization:")
        print(f"  Median: {diffry_median:.4f}°")
        print(f"  Range (normalized): {diffry_normalized.min():.4f}° to {diffry_normalized.max():.4f}°")

    # Mosaic intensity map (normalized phi and diffry)
    if mosa_diffry_values and mosa_intensity_values and mosaic_phi_values:
        phi_values = np.array(mosaic_phi_values)
        diffry_values = np.array(mosa_diffry_values)
        intensity_values = np.array(mosa_intensity_values)

        # Normalize phi and diffry
        phi_median = np.median(phi_values)
        diffry_median = np.median(diffry_values)
        
        phi_normalized = phi_values - phi_median
        diffry_normalized = diffry_values - diffry_median

        phi_min, phi_max = phi_normalized.min(), phi_normalized.max()
        diffry_min, diffry_max = diffry_normalized.min(), diffry_normalized.max()

        phi_grid, diffry_grid = np.meshgrid(
            np.linspace(phi_min, phi_max, 100),
            np.linspace(diffry_min, diffry_max, 100)
        )

        from scipy.interpolate import griddata
        intensity_grid = griddata(
            (phi_normalized, diffry_normalized),
            intensity_values,
            (phi_grid, diffry_grid),
            method='cubic'
        )

        fig, ax = plt.subplots(1, 1, figsize=(10, 8))
        contour = ax.contourf(phi_grid, diffry_grid, intensity_grid, levels=20, cmap=cm.viridis)
        
        plt.colorbar(contour, label='Intensity')
        ax.set_xlabel('χ-χb', fontsize=12, fontweight='semibold')
        ax.set_ylabel('ø-øb', fontsize=12, fontweight='semibold')
        #ax.set_title(f'Mosaic Intensity Map (Normalized)\nCentered at Chi={phi_median:.4f}°, Phi={diffry_median:.4f}°', 
                    #fontsize=13, fontweight='bold', pad=15)
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.set_facecolor('#F8F9FA')
        
        plt.savefig(f'{output_folder}/mosaic_intensity_3_map.png', dpi=300, bbox_inches='tight')
        log_message(f"Saved {output_folder}/mosaic_intensity_map.png")
        plt.close()
        
        print(f"\nMosaic map - Normalization:")
        print(f"  Phi median: {phi_median:.4f}°")
        print(f"  Phi range (normalized): {phi_min:.4f}° to {phi_max:.4f}°")
        print(f"  Diffry median: {diffry_median:.4f}°")
        print(f"  Diffry range (normalized): {diffry_min:.4f}° to {diffry_max:.4f}°")

    # Strain intensity map (normalized diffry)
    if strain_diffry_values and strain_intensity_values and strain_obpitch_values:
        obpitch_values = np.array(strain_obpitch_values)
        diffry_values = np.array(strain_diffry_values)
        intensity_values = np.array(strain_intensity_values)

        # Normalize diffry only
        diffry_median = np.median(diffry_values)
        diffry_normalized = diffry_values - diffry_median

        obpitch_min, obpitch_max = obpitch_values.min(), obpitch_values.max()
        diffry_min, diffry_max = diffry_normalized.min(), diffry_normalized.max()

        obpitch_grid, diffry_grid = np.meshgrid(
            np.linspace(obpitch_min, obpitch_max, 100),
            np.linspace(diffry_min, diffry_max, 100)
        )

        from scipy.interpolate import griddata
        intensity_grid = griddata(
            (obpitch_values, diffry_normalized),
            intensity_values,
            (obpitch_grid, diffry_grid),
            method='cubic'
        )

        fig, ax = plt.subplots(1, 1, figsize=(10, 8))
        contour = ax.contourf(obpitch_grid, diffry_grid, intensity_grid, levels=20, cmap=cm.viridis)
        
        plt.colorbar(contour, label='Intensity')
        ax.set_xlabel('Obpitch (degrees)', fontsize=12, fontweight='semibold')
        ax.set_ylabel('Diffry (normalized, degrees)', fontsize=12, fontweight='semibold')
        ax.set_title(f'Strain Intensity Map\nDiffry centered at {diffry_median:.4f}°', 
                    fontsize=13, fontweight='bold', pad=15)
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.set_facecolor('#F8F9FA')
        
        plt.savefig(f'{output_folder}/strain_intensity_map.png', dpi=300, bbox_inches='tight')
        log_message(f"Saved {output_folder}/strain_intensity_map.png")
        plt.show()
        
        print(f"\nStrain map - Diffry normalization:")
        print(f"  Diffry median: {diffry_median:.4f}°")
        print(f"  Diffry range (normalized): {diffry_min:.4f}° to {diffry_max:.4f}°")
        print(f"  Obpitch range: {obpitch_min:.4f}° to {obpitch_max:.4f}°")

plot_images_for_folder(data_1, output_folder='/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_remountedafterheating/Mg_remountedafterheating_farfield/Mosa_layer_extracted', start_frame=0, end_frame=30)

In [ ]:
#Does the same thing but you realised I normalised the phi and chi values so that they are both between 0-1
#Note: phi = diffry and chi = phi

def plot_images_for_folder(data, output_folder, start_frame=0, end_frame=31):
    frame_numbers = sorted(set([int(value['Frame No']) for key, value in data.items()]))
    
    if start_frame is not None:
        frame_numbers = [f for f in frame_numbers if f >= start_frame]
    if end_frame is not None:
        frame_numbers = [f for f in frame_numbers if f <= end_frame]
    
    num_columns = 5
    num_rows = (len(frame_numbers) + num_columns - 1) // num_columns
    fig, axs = plt.subplots(num_rows, num_columns, figsize=(num_columns * 5, num_rows * 5))
    fig.tight_layout(pad=3.0)
    axs = axs.flatten()

    rocking_diffry_values = []  
    rocking_intensity_values = [] 
    mosa_diffry_values = []  
    mosa_intensity_values = []  
    mosaic_phi_values = []
    strain_diffry_values = []  
    strain_intensity_values = [] 
    strain_obpitch_values = []

    for idx, frame_no in enumerate(frame_numbers):
        for file_name, value in data.items():
            if value['Frame No'] == str(frame_no):

                image_data = value['image_data']
                diffry = value.get('diffry', 'N/A')
                phi = value.get('phi', 'N/A')
                obpitch = value.get('obpitch', 'N/A')
                intensity = value['intensity']

                # roi
                # image_data = image_data[100:400, 100:400]
                intensity = np.max(image_data)

                if "rocking" in file_name.lower():
                    axs[idx].imshow(image_data, vmin=np.percentile(image_data, 1), vmax=np.percentile(image_data, 99))
                    axs[idx].set_title(f"Rocking - Frame: {frame_no}\nDiffry: {diffry}, Intensity: {intensity}")
                    rocking_diffry_values.append(diffry)  
                    rocking_intensity_values.append(intensity)  
                elif "mosa" in file_name.lower():
                    axs[idx].imshow(image_data, vmin=np.percentile(image_data, 5), vmax=np.percentile(image_data, 99.5))
                    axs[idx].set_title(f"Mosa - Frame: {frame_no}\nDiffry: {diffry}, Intensity: {intensity}, Phi: {phi}")
                    mosa_diffry_values.append(diffry)  
                    mosa_intensity_values.append(intensity)  
                    mosaic_phi_values.append(phi)
                elif "strain" in file_name.lower():
                    axs[idx].imshow(image_data, vmin=np.percentile(image_data, 5), vmax=np.percentile(image_data, 99.5))
                    axs[idx].set_title(f"Strain - Frame: {frame_no}\nDiffry: {diffry}, Intensity: {intensity}, Obpitch: {obpitch}")
                    strain_diffry_values.append(diffry)  
                    strain_intensity_values.append(intensity)  
                    strain_obpitch_values.append(obpitch)
                else:
                    log_message(f"Unknown file type: {file_name}")

                axs[idx].axis('off')
                break
        else:
            axs[idx].set_visible(False)
            print(f"No data returned for frame {frame_no}")

    for idx in range(len(frame_numbers), len(axs)):
        axs[idx].set_visible(False)

    frame_range = f"frames_{start_frame or 'start'}_to_{end_frame or 'end'}"
    plt.savefig(f'{output_folder}/all_{frame_range}.png', dpi=300)
    log_message(f"Saved frames subplot as {output_folder}/all_{frame_range}.png")
    plt.close()

    if rocking_diffry_values and rocking_intensity_values:
        fig, ax = plt.subplots(1, 1, figsize=(10, 5))
        ax.scatter(rocking_diffry_values, rocking_intensity_values, s=5, c='r') 
        ax.set_title('Rocking Curve')
        ax.set_xlabel('Diffry')
        ax.set_ylabel('Intensity')
        plt.savefig(f'{output_folder}/diffry_vs_intensity.png', dpi=300)
        log_message(f"Saved {output_folder}/diffry_vs_intensity.png")
        plt.close()

    if mosa_diffry_values and mosa_intensity_values and mosaic_phi_values:
        fig, ax = plt.subplots(1, 1, figsize=(10, 5))

        phi_values = np.array(mosaic_phi_values)
        diffry_values = np.array(mosa_diffry_values)
        intensity_values = np.array(mosa_intensity_values)

        phi_min, phi_max = phi_values.min(), phi_values.max()
        diffry_min, diffry_max = diffry_values.min(), diffry_values.max()

        phi_grid, diffry_grid = np.meshgrid(
            np.linspace(phi_min, phi_max, 100),
            np.linspace(diffry_min, diffry_max, 100)
        )

        from scipy.interpolate import griddata
        intensity_grid = griddata(
            (phi_values, diffry_values),
            intensity_values,
            (phi_grid, diffry_grid),
            method='cubic'
        )

        contour = ax.contourf(phi_grid, diffry_grid, intensity_grid, cmap=cm.viridis)
        plt.colorbar(contour, label='Intensity')
        ax.set_xlabel('Chi')
        ax.set_ylabel('Phi')
        plt.savefig(f'{output_folder}/mosaic_intensity_map.png', dpi=300)
        log_message(f"Saved {output_folder}/mosaic_intensity_map.png")
        plt.close()

    if strain_diffry_values and strain_intensity_values and strain_obpitch_values:
        fig, ax = plt.subplots(1, 1, figsize=(10, 5))

        obpitch_values = np.array(strain_obpitch_values)
        diffry_values = np.array(strain_diffry_values)
        intensity_values = np.array(strain_intensity_values)

        obpitch_min, obpitch_max = obpitch_values.min(), obpitch_values.max()
        diffry_min, diffry_max = diffry_values.min(), diffry_values.max()

        obpitch_grid, diffry_grid = np.meshgrid(
            np.linspace(obpitch_min, obpitch_max, 100),
            np.linspace(diffry_min, diffry_max, 100)
        )

        from scipy.interpolate import griddata
        intensity_grid = griddata(
            (obpitch_values, diffry_values),
            intensity_values,
            (obpitch_grid, diffry_grid),
            method='cubic'
        )

        contour = ax.contourf(obpitch_grid, diffry_grid, intensity_grid, cmap=cm.viridis)
        plt.colorbar(contour, label='Intensity')
        ax.set_xlabel('Obpitch')
        ax.set_ylabel('Diffry')
        plt.savefig(f'{output_folder}/strain_intensity_map.png', dpi=300)
        log_message(f"Saved {output_folder}/strain_intensity_map.png")
        plt.show()
plot_images_for_folder(data_1, output_folder = '/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_remountedafterheating/Mg_remountedafterheating_farfield/Mosa_layer_extracted' , start_frame=0, end_frame=30)
                       

In [ ]:
## Now we can do some image Analysis to select a specific frame number 
# Also we could do some hot pixel removal 

def get_image_data_for_frame(data, target_frame_no):
    
    for key, value in data.items():
        if value['Frame No'] == target_frame_no:
            diffry = value['diffry']
            intensity = value['intensity']
            ob_pitch = value['obpitch']
            chi = value['chi']
            print(f"Frame No: {target_frame_no}, Diffry: {diffry}, Intensity: {intensity}, Ob Pitch: {ob_pitch}, Chi: {chi}")
            return value['image_data'], diffry, intensity, ob_pitch, chi
        
            
    return None

# image processing pipeline

def image_processing(image_data):

    def hot_pixel_removal(image, perform: bool, ksize=3):

        if perform is True:

            if image.dtype != np.uint8 and image.dtype != np.float32:
                image = image.astype(np.float32)

            median = cv2.medianBlur(image, ksize)

            subtracted_image = np.subtract(image, median)

            threshold = np.std(subtracted_image) # I am not sure if this is the best threshold

            hot_pixels = subtracted_image > threshold
            
            image[hot_pixels] = median[hot_pixels]

            return image
        
        else:
            return image

    # remove ceil and floor values
    # NOTE: this is making noise in the image. I will have to do more research on this

    def remove_ceil_floor(image: np.ndarray, perform: bool, ceil_frac: float) -> np.ndarray:

        if perform is True:
            ceil = np.quantile(image, ceil_frac)
            floor = np.quantile(image, 1 - ceil_frac)

            image[image > ceil] = ceil
            image[image < floor] = floor

            return image
        
        else:
            return image
        
    # remove the hot pixels

    image_processed = hot_pixel_removal(image_data, perform=True, ksize=5) 


    # remove the ceil and floor values

    image_processed = remove_ceil_floor(image_processed, perform=True, ceil_frac=0.99)

    return image_processed

# plot the image data for a specific frame number

def plot_image_for_frame(data, target_frame_no, output_folder):

    image_data, diffry, intensity, obpitch, chi = get_image_data_for_frame(data, target_frame_no)

    if image_data is not None:
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        image_data = image_processing(image_data)
        ax.imshow(image_data, vmin = np.percentile(image_data, 5), vmax = np.percentile(image_data, 99.5))    
        ax.set_title(f"Frame No: {target_frame_no}, Diffry: {diffry}, Intensity: {intensity}")
        # plt.savefig(f'{output_folder}/frame_{target_frame_no}.png', dpi=300)
        plt.imshow(image_data, cmap='gray')
        # log_message(f"Saved {output_folder}/frame_{target_frame_no}.png")
        plt.show()
    else:
        log_message(f"Frame {target_frame_no} not found")

## Selecting Region of Interest

In [ ]:
## selecting a region of interest or cropping out interesting regions in the sample 

def plot_image_for_frame(data, target_frame_no, output_folder, roi=None):
    """
    Plot image data for a specific frame with optional region of interest.
    
    Parameters:
    -----------
    data : dataset
        Input data containing image frames
    target_frame_no : int
        Frame number to plot
    output_folder : str
        Output directory path
    roi : dict or tuple, optional
        Region of interest specification. Can be:
        - dict: {'x_min': int, 'x_max': int, 'y_min': int, 'y_max': int}
        - tuple: (x_min, x_max, y_min, y_max)
        - None: plots full image (default)
    """
    
    image_data, diffry, intensity, obpitch, chi = get_image_data_for_frame(data, target_frame_no)
    
    if image_data is not None:
        # Process the full image first
        image_data = image_processing(image_data)
        
        # Extract ROI if specified
        if roi is not None:
            if isinstance(roi, dict):
                x_min = roi.get('x_min', 0)
                x_max = roi.get('x_max', image_data.shape[1])
                y_min = roi.get('y_min', 0)
                y_max = roi.get('y_max', image_data.shape[0])
            elif isinstance(roi, tuple) and len(roi) == 4:
                x_min, x_max, y_min, y_max = roi
            else:
                raise ValueError("ROI must be a dict or tuple of 4 values")
            
            # Validate ROI bounds
            x_min = max(0, x_min)
            y_min = max(0, y_min)
            x_max = min(image_data.shape[1], x_max)
            y_max = min(image_data.shape[0], y_max)
            
            # Crop to ROI
            image_data_roi = image_data[y_min:y_max, x_min:x_max]
            title_suffix = f" | ROI: [{x_min}:{x_max}, {y_min}:{y_max}]"
        else:
            image_data_roi = image_data
            title_suffix = ""
        
        # Create figure
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        
        # Calculate percentiles on ROI data
        vmin = np.percentile(image_data_roi, 5)
        vmax = np.percentile(image_data_roi, 99.5)
        
        # Display image
        ax.imshow(image_data_roi, cmap='gray', vmin=vmin, vmax=vmax)
        ax.set_title(f"Frame No: {target_frame_no}, Diffry: {diffry}, Intensity: {intensity}{title_suffix}")
        
        # Optional: save figure
        # plt.savefig(f'{output_folder}/frame_{target_frame_no}_roi.png', dpi=300, bbox_inches='tight')
        # log_message(f"Saved {output_folder}/frame_{target_frame_no}_roi.png")
        
        plt.show()
    else:
        log_message(f"Frame {target_frame_no} not found")


# Example we could
Full image
plot_image_for_frame(data, 100, output_folder)

#With ROI as dictionary
plot_image_for_frame(data, 100, output_folder, roi={'x_min': 200, 'x_max': 800, 'y_min': 150, 'y_max': 750})

#With ROI as tuple
plot_image_for_frame(data, 100, output_folder, roi=(200, 800, 150, 750))

In [ ]:
# This a robust function I created to create 1D COM for both phi and chi, and  2DCOM 
# with combined colour bar showing chi and phi

#N.B: phi = diffry and chi = phi
# I am still not convince about the 2D COM plot, we might have to modify the code.


import scipy.ndimage
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.cm as cm

# Seems jupyter can't read latex. Turning off latex
plt.rcParams["text.usetex"] = False  # Turn off LaTeX

def com_edf_2d(data, output_folder):
    diffry_list = []
    chi_list = []  # phi values
    image_stack = []

    for item in data.values():
        diffry_list.append(item['diffry'])
        chi_list.append(item['phi'])
        image_stack.append(item['image_data'])

    # Apply noise floor
    for i, frame in enumerate(image_stack):
        noise_floor = 150
        image_stack[i][image_stack[i] < noise_floor] = 0

    # Apply Gaussian filter
    for i, frame in enumerate(image_stack):
        sigma = 3
        image_stack[i] = scipy.ndimage.gaussian_filter(frame, sigma)

    # Convert to numpy array
    image_stack = np.array(image_stack)
    
    # Find max indices along stack dimension
    max_indices = np.argmax(image_stack, axis=0)
    
    # Convert to arrays
    diffry_array = np.array(diffry_list).flatten()
    chi_array = np.array(chi_list).flatten()
    
    # Normalize diffry and chi (center at median)
    diffry_median = np.median(diffry_array)
    chi_median = np.median(chi_array)
    
    diffry_normalized = diffry_array - diffry_median
    chi_normalized = chi_array - chi_median
    
    # Create maps based on max indices
    diffry_map = diffry_normalized[max_indices]
    chi_map = chi_normalized[max_indices]
    
#     # Apply smoothing to the maps
#     smooth_sigma = max(diffry_map.shape) / 100  # Adaptive sigma
#     diffry_map_smooth = scipy.ndimage.gaussian_filter(diffry_map, sigma=smooth_sigma)
#     chi_map_smooth = scipy.ndimage.gaussian_filter(chi_map, sigma=smooth_sigma)
    
    # Normalize to 0-1 range
    diffry_min, diffry_max = np.percentile(diffry_map, [5, 95])
    chi_min, chi_max = np.percentile(chi_map, [5, 95])
    
    diffry_norm = np.clip((diffry_map - diffry_min) / (diffry_max - diffry_min), 0, 1)
    chi_norm = np.clip((chi_map - chi_min) / (chi_max - chi_min), 0, 1)
    
    # Create combined map using viridis colormap
    # Combine diffry and chi into a single value (e.g., magnitude or average)
    # Option 1: Magnitude
    combined_magnitude = np.sqrt(diffry_map**2 + chi_map**2)
    
    # Option 2: Weighted combination for viridis mapping
    # We'll use angle in 2D space to map to viridis
    angle_map = np.arctan2(chi_map, diffry_map)  # Range: -π to π
    angle_normalized = (angle_map + np.pi) / (2 * np.pi)  # Normalize to 0-1
    
    # Option 3: Use magnitude with viridis
    magnitude_map = np.sqrt(diffry_map**2 + chi_map**2)
    
    # Get viridis colormap
    viridis = cm.get_cmap('RdBu') #Replace with the color map of choice
    
    
    # Apply viridis to the combined data
    # Using angle for hue-like representation
    viridis_image_angle = viridis(angle_normalized)[:, :, :3]  # Remove alpha channel
    
    # Using magnitude
    magnitude_norm = (magnitude_map - magnitude_map.min()) / (magnitude_map.max() - magnitude_map.min())
    viridis_image_mag = viridis(magnitude_norm)[:, :, :3]
    
    # **MODIFIED: Set fixed colorbar range for combined plot**
    fixed_min = -0.3
    fixed_max = 0.3
    
    # Create 2D colorbar for viridis (showing the angle mapping)
    n_colors = 256
    angle_cbar = np.linspace(0, 1, n_colors)
    angle_cbar_2d = np.tile(angle_cbar, (n_colors, 1))
    viridis_cbar = viridis(angle_cbar_2d)[:, :, :3]
    
    # **MODIFIED: Calculate colorbar values with fixed range**
    diffry_cbar_range = np.linspace(fixed_min, fixed_max, n_colors)
    chi_cbar_range = np.linspace(fixed_min, fixed_max, n_colors)
    diffry_cbar_grid, chi_cbar_grid = np.meshgrid(diffry_cbar_range, chi_cbar_range)
    angle_cbar_actual = np.arctan2(chi_cbar_grid, diffry_cbar_grid)
    angle_cbar_norm = (angle_cbar_actual + np.pi) / (2 * np.pi)
    viridis_cbar_2d = viridis(angle_cbar_norm)[:, :, :3]
    
    # Create 3-panel plot with proper colorbar
    fig = plt.figure(figsize=(24, 6))
    gs = GridSpec(1, 4, width_ratios=[1, 1, 1, 0.15], wspace=0.3)
    
    # Plot 1: Diffry (φ) map
    ax1 = fig.add_subplot(gs[0])
    cax1 = ax1.imshow(diffry_map, cmap='RdBu', 
                     vmin=np.percentile(diffry_normalized, 5), 
                     vmax=np.percentile(diffry_normalized, 95))
    cbar1 = plt.colorbar(cax1, ax=ax1)
    cbar1.set_label('φ - φ₀ (degrees)', fontsize=12)
    ax1.set_title(f'Diffry (φ) Center of Mass Map\n(Centered at φ₀={diffry_median:.4f}°)', 
                 fontsize=12, fontweight='bold')
    ax1.set_xlabel('X pixel', fontsize=11)
    ax1.set_ylabel('Y pixel', fontsize=11)
    
    # Plot 2: Chi (Phi) map
    ax2 = fig.add_subplot(gs[1])
    cax2 = ax2.imshow(chi_map, cmap='RdBu', 
                     vmin=np.percentile(chi_normalized, 5), 
                     vmax=np.percentile(chi_normalized, 95))
    cbar2 = plt.colorbar(cax2, ax=ax2)
    cbar2.set_label('χ - χ₀ (degrees)', fontsize=12)
    ax2.set_title(f'Chi (χ) Center of Mass Map\n(Centered at χ₀={chi_median:.4f}°)', 
                 fontsize=12, fontweight='bold')
    ax2.set_xlabel('X pixel', fontsize=11)
    ax2.set_ylabel('Y pixel', fontsize=11)
    
    # Plot 3: Combined viridis plot (using angle)
    ax3 = fig.add_subplot(gs[2])
    ax3.imshow(viridis_image_angle)
    ax3.set_title('Combined φ and χ Map (plasma)\nColor = Direction(φ, χ)', 
                 fontsize=12, fontweight='bold')
    ax3.set_xlabel('X pixel', fontsize=11)
    ax3.set_ylabel('Y pixel', fontsize=11)
    
    # **MODIFIED: Plot 4: 2D Colorbar with SQUARE aspect ratio**
    ax_cbar = fig.add_subplot(gs[3])
    ax_cbar.imshow(viridis_cbar_2d, origin='lower', 
                   extent=[fixed_min, fixed_max, fixed_min, fixed_max], 
                   aspect='equal')  # Changed from 'auto' to 'equal'
    ax_cbar.set_xlabel('φ-φ₀ (°)', fontsize=10, fontweight='semibold')
    ax_cbar.set_ylabel('χ-χ₀ (°)', fontsize=10, fontweight='semibold')
    ax_cbar.set_title('Color\nReference', fontsize=10, fontweight='bold')
    
    # **NEW: Set matching tick intervals for both axes**
    tick_interval = 0.1  # Adjust this value as needed (e.g., 0.05, 0.1, 0.15)
    ticks = np.arange(fixed_min, fixed_max + tick_interval, tick_interval)
    ax_cbar.set_xticks(ticks)
    ax_cbar.set_yticks(ticks)
    ax_cbar.tick_params(labelsize=9)
    
    plt.savefig(f'{output_folder}/2D_com_maps_plasma_final.png', dpi=300, bbox_inches='tight')
    log_message(f"Saved {output_folder}/2D_com_maps_plasma.png")
    plt.show()
    
    # Create individual high-res plots
    # Diffry only
    fig, ax = plt.subplots(figsize=(10, 8))
    cax = ax.imshow(diffry_map, cmap='RdBu', 
                    vmin=np.percentile(diffry_normalized, 5), 
                    vmax=np.percentile(diffry_normalized, 95))
    cbar = fig.colorbar(cax)
    cbar.set_label('φ - φ₀ (degrees)', fontsize=12)
    ax.set_title(f'Diffry (φ) Center of Mass Map (Smoothed)\n(Centered at φ₀={diffry_median:.4f}°)', 
                fontsize=14, fontweight='bold')
    ax.set_xlabel('X pixel', fontsize=11)
    ax.set_ylabel('Y pixel', fontsize=11)
    plt.savefig(f'{output_folder}/plasma_1D_com_diffry6_final.png', dpi=300, bbox_inches='tight')
    log_message(f"Saved {output_folder}/1D_com_diffry.png")
    plt.show()
    plt.close()
    
    # Chi only
    fig, ax = plt.subplots(figsize=(10, 8))
    cax = ax.imshow(chi_map, cmap='RdBu', 
                    vmin=np.percentile(chi_normalized, 5), 
                    vmax=np.percentile(chi_normalized, 95))
    cbar = fig.colorbar(cax)
    cbar.set_label('χ - χ₀ (degrees)', fontsize=12)
    ax.set_title(f'Chi (χ) Center of Mass Map (Smoothed)\n(Centered at χ₀={chi_median:.4f}°)', 
                fontsize=14, fontweight='bold')
    ax.set_xlabel('X pixel', fontsize=11)
    ax.set_ylabel('Y pixel', fontsize=11)
    plt.savefig(f'{output_folder}/plasma_1D_com_chi6_final.png', dpi=300, bbox_inches='tight')
    log_message(f"Saved {output_folder}/1D_com_chi.png")
    plt.show()
    plt.close()
    
    # **MODIFIED: Combined viridis with SQUARE 2D colorbar (high-res standalone)**
    fig = plt.figure(figsize=(14, 10))
    gs = GridSpec(1, 2, width_ratios=[5, 1], wspace=0.2)
    
    # Main plot
    ax_main = fig.add_subplot(gs[0])
    ax_main.imshow(viridis_image_angle)
    ax_main.set_title(f'Combined φ and χ Center of Mass Map (Plasma)\nColor represents direction in (φ, χ) space\nφ₀={diffry_median:.4f}°, χ₀={chi_median:.4f}°', 
                fontsize=15, fontweight='bold', pad=15)
    ax_main.set_xlabel('X pixel', fontsize=12)
    ax_main.set_ylabel('Y pixel', fontsize=12)
    
    # **MODIFIED: 2D Colorbar with SQUARE aspect ratio**
    ax_cbar = fig.add_subplot(gs[1])
    ax_cbar.imshow(viridis_cbar_2d, origin='lower', 
                   extent=[fixed_min, fixed_max, fixed_min, fixed_max], 
                   aspect='equal')  # Changed from 'auto' to 'equal'
    ax_cbar.set_xlabel('φ - φ₀ (degrees)', fontsize=11, fontweight='semibold')
    ax_cbar.set_ylabel('χ - χ₀ (degrees)', fontsize=11, fontweight='semibold')
    ax_cbar.set_title('Color Map\nReference', fontsize=12, fontweight='bold', pad=10)
    
    # **NEW: Set matching tick intervals for both axes**
    ax_cbar.set_xticks(ticks)
    ax_cbar.set_yticks(ticks)
    ax_cbar.tick_params(labelsize=10)
    
    # Add grid to colorbar for easier reading
    #ax_cbar.grid(True, alpha=0.3, color='white', linewidth=0.5)
    
    plt.savefig(f'{output_folder}/3combined_com_plasma.png', dpi=300, bbox_inches='tight')
    log_message(f"Saved {output_folder}/combined_com_viridis.png")
    plt.show()
    
results = com_edf_2d(data_1, output_folder='/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_remountedafterheating/Mg_remountedafterheating_farfield/Mosa_layer_extracted')

### In case we need to work with H5 files 

In [ ]:
##Converting all EDF files to H5 files using darfix and silx
## The converted H5 structure might not be the same as an actual H5 structure, but hey it works!
#Also it easier to work with H5 files 

import subprocess
import glob
import os
import re

base_dir = '/scratch/groups/leoradm/edemdh/esrf_202204/id06-hxm/Mg_remountedafterheating/Mg_remountedafterheating_farfield/Mosa_layer_extracted/'

# Define output directory for all H5 files
output_dir = os.path.join(base_dir, 'Converted_Remounted_After_Heating')
os.makedirs(output_dir, exist_ok=True)

# Find all subdirectories
all_items = os.listdir(base_dir)
folders = []

for item in all_items:
    full_path = os.path.join(base_dir, item)
    # Check if it's a directory and matches pattern
    if os.path.isdir(full_path) and item.startswith('mosalayer_10x_'):
        folders.append(full_path)

print(f"Found {len(folders)} matching folders:\n")
for folder in sorted(folders):
    print(f"  - {os.path.basename(folder)}")
print(f"\nAll H5 files will be saved to: {output_dir}\n")

# Process each folder
for folder in sorted(folders):
    folder_name = os.path.basename(folder)
    print(f"Processing: {folder_name}")
    
    # Find EDF files
    edf_pattern = os.path.join(folder, '*.edf')
    edf_files = glob.glob(edf_pattern)
    
    print(f"  Found {len(edf_files)} EDF files")
    
    if edf_files:
        # Save to central output directory instead of source folder
        output_file = os.path.join(output_dir, f'{folder_name}_converted.h5')
        cmd = ['silx', 'convert'] + edf_files + ['-o', output_file, '--mode', 'w']
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            size = os.path.getsize(output_file)
            print(f"  ✓ Success! {size / (1024**2):.2f} MB")
            print(f"    Saved to: {output_file}\n")
        else:
            print(f"  ✗ Failed")
            print(f"    Error: {result.stderr}\n")
    else:
        print(f"  ⊘ Skipped (no EDF files)\n")

print(f"Done! All H5 files are in: {output_dir}")

## Plotting 3D volume with converted H5 files 

In [ ]:
## Still working on this code but I think it works for now with the converted H5 files 
# Better to run this code with vscode to avoid to many debugging 

import os, sys, h5py, hdf5plugin, glob, numpy as np
from typing import List

try:
    import cupy as cp; xp, GPU = cp, True
    print("CuPy found – running on GPU")
except ImportError:
    import numpy as xp; GPU = False
    print("CuPy not found – falling back to NumPy (CPU)")

import napari
from magicgui import magicgui
from magicgui.widgets import Label, PushButton, LineEdit
from napari.qt.threading import thread_worker
from qtpy.QtCore    import Qt, QTimer
from qtpy.QtWidgets import (QApplication, QWidget, QListWidget,
                            QListWidgetItem, QSpinBox, QLabel,
                            QHBoxLayout, QVBoxLayout, QFileDialog)

def launch_viewer(root_dir, motors_file):
    root_dir = root_dir.rstrip('/') + '/'
    
    # Fix: Use proper glob pattern to find actual files
    scan_pattern = root_dir + "scan*/DEPow_0.000000_rockinglayer_10x_*_converted.h5"
    scan_files = sorted(glob.glob(scan_pattern))
    
    if not scan_files:
        raise RuntimeError(f"No scan files found matching pattern: {scan_pattern}")
    
    print(f"Found {len(scan_files)} scan files:")
    for f in scan_files[:3]:  # Print first 3 as preview
        print(f"  {f}")
    
    # Open the FIRST actual file found (not the pattern)
    with h5py.File(scan_files[0]) as f0:
        SCAN_COUNT, H, W = f0['scan_0/image/data'].shape

    with h5py.File(motors_file) as f:
        phi_vals = np.round(f['scan_0/instrument/positioners/diffry'][:], 4)
        chi_vals = np.round(f['scan_0/instrument/positioners/chi'][:], 4)

    PHI_COUNT = np.unique(phi_vals).size
    CHI_COUNT = np.unique(chi_vals).size

    def _frame(path: str, χ: int, φ: int) -> np.ndarray:
        χ = np.clip(χ, 0, CHI_COUNT-1)
        φ = np.clip(φ, 0, PHI_COUNT-1)
        g = φ*CHI_COUNT + χ
        with h5py.File(path) as f:
            return f['scan_0/image/data'][g]

    def _mean(path: str, χ0: int, φ0: int, χr: int, φr: int) -> np.ndarray:
        acc, cnt = None, 0
        for dχ in range(-χr, χr+1):
            for dφ in range(-φr, φr+1):
                f = _frame(path, χ0+dχ, φ0+dφ)
                acc = f if acc is None else acc + f
                cnt += 1
        return acc / cnt

    current_chi = 0
    current_phi = 42
    current_chi_rad = 0
    current_phi_rad = 0
    local_chi_off = np.zeros(len(scan_files), dtype=int)
    local_phi_off  = np.zeros(len(scan_files), dtype=int)
    slice_visible = np.ones(len(scan_files), dtype=bool)

    @thread_worker
    def _volume_worker(χ, φ, χr, φr, χoff, φoff, vis):
        vol = []
        for i, path in enumerate(scan_files):
            if not vis[i]:
                vol.append(np.zeros((H, W), dtype=np.float32)); continue
            eff_χ = χ + int(χoff[i])
            eff_φ = φ + int(φoff[i])
            vol.append(_mean(path, eff_χ, eff_φ, χr, φr))
        return np.log10(np.stack(vol).astype(np.float32))

    first_vol = _volume_worker.__wrapped__(
        current_chi, current_phi, current_chi_rad, current_phi_rad,
        local_chi_off, local_phi_off, slice_visible
    )
    vmin, vmax = float(first_vol.min()), float(first_vol.max())
    dz, dx, dy = 5.0, 0.15, 0.15

    viewer = napari.Viewer(title="DFXM volume (χ, φ)")
    vol_layer = viewer.add_image(
        first_vol, name='intensity', contrast_limits=(vmin, vmax),
        colormap='viridis', rendering='mip',
        iso_threshold=vmin + 0.25*(vmax - vmin), attenuation=0.01,
        scale=(dz, dy, dx)
    )
    chi_phi_label = Label()

    def _status():
        g = np.clip(current_phi,0,PHI_COUNT-1)*CHI_COUNT + np.clip(current_chi,0,CHI_COUNT-1)
        chi_phi_label.value = f"χ = {chi_vals[g]:.3f}°,   φ = {phi_vals[g]:.3f}°"
        viewer.status = f"χ‑idx {current_chi}, φ‑idx {current_phi} · "+chi_phi_label.value

    def _refresh():
        worker = _volume_worker(
            current_chi, current_phi, current_chi_rad, current_phi_rad,
            local_chi_off.copy(), local_phi_off.copy(), slice_visible.copy()
        )
        worker.returned.connect(lambda d: setattr(vol_layer, "data", d))
        worker.returned.connect(lambda _: _status())
        worker.start()

    @magicgui(χ_idx={'widget_type':'Slider','min':0,'max':CHI_COUNT-1,
                     'orientation':'horizontal','tracking':False},
              auto_call=True)
    def χ_slider(χ_idx: int = 0):
        nonlocal current_chi; current_chi = int(χ_idx); _refresh()

    @magicgui(φ_idx={'widget_type':'Slider','min':0,'max':PHI_COUNT-1,
                     'orientation':'horizontal','tracking':False},
              auto_call=True)
    def φ_slider(φ_idx: int = 0):
        nonlocal current_phi; current_phi = int(φ_idx); _refresh()

    @magicgui(χ_rad={'widget_type':'SpinBox','min':0,'max':CHI_COUNT//2},
              auto_call=True, layout='horizontal')
    def χ_comb(χ_rad: int = 0):
        nonlocal current_chi_rad; current_chi_rad = int(χ_rad); _refresh()

    @magicgui(φ_rad={'widget_type':'SpinBox','min':0,'max':PHI_COUNT//2},
              auto_call=True, layout='horizontal')
    def φ_comb(φ_rad: int = 0):
        nonlocal current_phi_rad; current_phi_rad = int(φ_rad); _refresh()

    gbox = QWidget(); g_lay = QVBoxLayout(gbox)
    for w in (χ_slider.native, φ_slider.native,
              χ_comb.native, φ_comb.native, chi_phi_label.native):
        g_lay.addWidget(w)

    class SliceCtrl(QWidget):
        def __init__(self):
            super().__init__()
            lay = QVBoxLayout(self)
            self.list = QListWidget()
            self.list.setSelectionMode(QListWidget.MultiSelection)
            for i in range(len(scan_files)):
                item = QListWidgetItem(f"z {i}")
                item.setFlags(item.flags() | Qt.ItemIsUserCheckable)
                item.setCheckState(Qt.Checked)
                self.list.addItem(item)
            lay.addWidget(self.list)

            rowχ = QHBoxLayout()
            rowχ.addWidget(QLabel("Δχ")); self.chi_sp = QSpinBox()
            self.chi_sp.setRange(-CHI_COUNT, CHI_COUNT); rowχ.addWidget(self.chi_sp)
            lay.addLayout(rowχ)

            rowφ = QHBoxLayout()
            rowφ.addWidget(QLabel("Δφ")); self.phi_sp = QSpinBox()
            self.phi_sp.setRange(-PHI_COUNT, PHI_COUNT); rowφ.addWidget(self.phi_sp)
            lay.addLayout(rowφ)

            self.list.itemSelectionChanged.connect(self._sync)
            self.list.itemChanged.connect(self._vis_change)
            self.chi_sp.valueChanged.connect(self._apply)
            self.phi_sp.valueChanged.connect(self._apply)

        def _rows(self): return [i.row() for i in self.list.selectedIndexes()]

        def _sync(self):
            if not self._rows(): return
            r = self._rows()[0]
            self.chi_sp.blockSignals(True); self.phi_sp.blockSignals(True)
            self.chi_sp.setValue(int(local_chi_off[r]))
            self.phi_sp.setValue(int(local_phi_off[r]))
            self.chi_sp.blockSignals(False); self.phi_sp.blockSignals(False)

        def _apply(self):
            for r in self._rows():
                local_chi_off[r] = self.chi_sp.value()
                local_phi_off[r]  = self.phi_sp.value()
            _refresh()

        def _vis_change(self, item):
            row = self.list.row(item)
            slice_visible[row] = (item.checkState()==Qt.Checked)
            _refresh()

    local_box = SliceCtrl()

    class FileBox(QWidget):
        def __init__(self):
            super().__init__()
            lay = QVBoxLayout(self)
            self.root_edit   = LineEdit(value=root_dir, label="Root folder")
            self.motor_edit  = LineEdit(value=motors_file, label="Motors .h5")
            browse_btn = PushButton(text="Browse folder…")
            motor_btn  = PushButton(text="Browse file…")
            apply_btn  = PushButton(text="Apply & Reload")

            row1 = QHBoxLayout(); row1.addWidget(self.root_edit.native)
            row1.addWidget(browse_btn.native); lay.addLayout(row1)
            row2 = QHBoxLayout(); row2.addWidget(self.motor_edit.native)
            row2.addWidget(motor_btn.native); lay.addLayout(row2)
            lay.addWidget(apply_btn.native)

            browse_btn.clicked.connect(self._pick_folder)
            motor_btn .clicked.connect(self._pick_file)
            apply_btn .clicked.connect(self._apply)

        def _pick_folder(self):
            d = QFileDialog.getExistingDirectory(self, "Select scan folder", root)
            if d: self.root_edit.value = d if d.endswith(os.sep) else d+os.sep

        def _pick_file(self):
            f, _ = QFileDialog.getOpenFileName(self, "Select motors HDF5",
                                               os.path.dirname(motors_file),
                                               "HDF5 files (*.h5)")
            if f: self.motor_edit.value = f

        def _apply(self):
            new_root  = self.root_edit.value
            new_motor = self.motor_edit.value
            viewer.close()
            QTimer.singleShot(0, lambda: launch_viewer(new_root, new_motor))

    file_box = FileBox()

    viewer.window.add_dock_widget(local_box,name="LOCAL slice tools", area="right")
    viewer.window.add_dock_widget(gbox,     name="GLOBAL controls", area="right")
    viewer.window.add_dock_widget(file_box, name="FILE handling", area="right")

    app = QApplication.instance()
    app.setStyleSheet(app.styleSheet()+"QLabeledSlider > QAbstractSpinBox {padding:0px 4px;}")

    _status()


    from napari_animation import Animation

    animation = Animation(viewer)

    viewer.dims.ndisplay = 3
    viewer.camera.angles = (0.0, 0.0, 90.0)
    animation.capture_keyframe()
    viewer.camera.zoom = 2.4
    animation.capture_keyframe()
    viewer.camera.angles = (-7.0, 15.7, 62.4)
    animation.capture_keyframe(steps=60)
    viewer.camera.angles = (2.0, -24.4, -36.7)
    animation.capture_keyframe(steps=60)
    viewer.reset_view()
    viewer.camera.angles = (0.0, 0.0, 90.0)
    animation.capture_keyframe()
    animation.animate('demo.mov', canvas_only=True)

    viewer.show()

if __name__ == "__main__":
    init_root   = "/Users/edemdoehonu/Library/CloudStorage/GoogleDrive-edemdh@stanford.edu/My Drive/Morning_after_H5files/"
    init_motors = init_root + "DEPow_0.000000_rockinglayer_10x_00_converted.h5"
    launch_viewer(init_root, init_motors)
    napari.run()

